In [3]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import os, sys, inspect

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
from urllib.request import Request, urlopen
from urllib.parse import quote

import re

commons_site = pb.Site("commons", "commons")

In [4]:
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

req_proxy = RequestProxy()

2017-12-30 22:33:42,623 root   DEBUG    === Initialized Proxy Parsers ===
2017-12-30 22:33:42,627 root   DEBUG    	 FreeProxy Parser of 'http://free-proxy-list.net' with required bandwidth: '5' KBs
2017-12-30 22:33:42,629 root   DEBUG    	 ProxyForEU Parser of 'http://proxyfor.eu/geo.php' with required bandwidth: '1.0' KBs
2017-12-30 22:33:42,631 root   DEBUG    	 RebroWeebly Parser of 'http://rebro.weebly.com' with required bandwidth: '5' KBs
2017-12-30 22:33:42,633 root   DEBUG    	 SemairProxy Parser of 'https://premproxy.com/list/' with required bandwidth: '5' KBs
2017-12-30 22:33:42,634 root   DEBUG    =================================
2017-12-30 22:33:43,223 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x7f366c0a9608>
2017-12-30 22:33:43,568 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2017-12-30 22:33:43,570 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    P

In [19]:
url = 'http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/presidente/Paginas/2014/170614-galeriareyes.aspx'
excluded = []
categories = ['Palacio de la Zarzuela', 'June 2014 in Madrid', 'Mariano Rajoy in 2014']
cat_string = '\n'.join(['[[Category:'+i+']]' for i in categories])

In [8]:
while True:
    request = req_proxy.generate_proxied_request(url)
    if request is not None:
        #print("{0}".format(u''.join(request.text).encode('utf-8')))
        response = request.text
        break
    else :
        print ('Trying')
soup = BeautifulSoup(response, 'html.parser')

2017-12-30 22:34:46,890 root   DEBUG    Using proxy: 221.120.214.174:8080 | Samair


RR Status 200


In [9]:
title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
#title = u'Segundo Gobierno de Rodríguez Zapatero (IX Legislatura) 2010'
title

'Rajoy junto a los Reyes y los titulares de los poderes del Estado'

In [10]:
import dateparser as dp
import datetime

try:
    pub_date_sp = ' '.join(soup.find_all("div", class_="contenidoFecha")[0].get_text().strip().split(' ')[-4:])
    parsed_pub_date = dp.parse(pub_date_sp)
    pub_date = parsed_pub_date.strftime('%Y-%m-%d')
except :
    pub_date = None

pub_date

'2014-06-30'

In [11]:
import re
p = re.compile("gFIO\[\d{1,2}\] = \\'(.*)\\';")
image_list = ['http://www.lamoncloa.gob.es' + url for url in re.findall(p, response)]
image_list

['http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2014/31BF-EDAC-AR4.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2014/496B-2C8E-AR3.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2014/B864-D29A-AR2.jpg']

In [12]:
p = re.compile("gFT\[\d{1,2}\] = \\'(.*)\\';")
description_list = [desc.strip() for desc in re.findall(p, response)]
description_list

['El presidente del Gobierno, Mariano Rajoy, saluda al Rey don Juan Carlos, durante el almuerzo que Sus Majestades los Reyes han ofrecido a los titulares de los poderes del Estado.',
 'El presidente del Gobierno, Mariano Rajoy, y su esposa conversan con Sus Majestades los Reyes durante el almuerzo que estos han ofrecido en el Palacio de la Zarzuela a los titulares de los poderes del Estado.',
 'Don Juan Carlos y Doña Sofía posan con el presidente del Gobierno, Mariano Rajoy, y su esposa; el presidente del Senado, Pío García-Escudero; el presidente del Tribunal Constitucional, Francisco Pérez de los Cobos; y del presidente del Tribunal Supremo y del Consejo General del Poder Judicial, Carlos Lesmes, y su esposa.']

In [13]:
p = re.compile("gFA\[\d{1,2}\] = \\'(.*)\\';")
author_list = [author.strip() for author in re.findall(p, response)]
author_list

['Pool Moncloa', 'Pool Moncloa', 'Pool Moncloa']

In [14]:
truth_list = ['Pool' in author for author in author_list]
truth_list

[True, True, True]

In [15]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Galería fotográfica: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

In [ ]:
from PIL import Image
from io import BytesIO
import requests

used_names = []
use_filename = []

counter = 0
for i, image in enumerate(image_list):
    if i in excluded or truth_list[i] == False:
        continue
    href = quote(image.encode('utf-8'), ':/')
    image_name = "%s %02d.jpg" %(title, counter)
    print (image_name)
    counter+=1
    image_path = os.path.join(os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0])), image_name)
    while True:
        print ('Trying')
        img_request = req_proxy.generate_proxied_request(href)
        if img_request is not None:
            bytes = BytesIO(img_request.content)
            image = Image.open(bytes)
            image.save(image_path)
            print ('Accessed. Going to next image')
            break
        else :
            print ('Still trying')
    vars = {
        "url": url,
        "description": description_list[i],
        "author": author_list[i],
        "date": pub_date,
        "title": title,
        "cat_string": cat_string
    }
    t = Template(template)
    _text = t.render(**vars)
    print(_text)
    
    bot = UploadRobot(image_path,
                          description = _text,
                          useFilename = image_name,
                          keepFilename = True,
                          verifyDescription = False,
                          ignoreWarning = True,
                          targetSite = commons_site)
    bot.run()
        
    os.remove(image_path) 

2017-12-30 22:40:10,590 root   DEBUG    Using proxy: 93.142.186.171:8080 | Samair


Rajoy junto a los Reyes y los titulares de los poderes del Estado 00.jpg
Trying


2017-12-30 22:40:15,008 pywiki WARNING  url as string is deprecated. Use an iterable instead.


RR Status 200
Accessed. Going to next image
=={{int:filedesc}}==
{{Information
|description={{es|1=El presidente del Gobierno, Mariano Rajoy, saluda al Rey don Juan Carlos, durante el almuerzo que Sus Majestades los Reyes han ofrecido a los titulares de los poderes del Estado.}}
|date=2014-06-30
|source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/presidente/Paginas/2014/170614-galeriareyes.aspx Galería fotográfica: Rajoy junto a los Reyes y los titulares de los poderes del Estado]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Palacio de la Zarzuela]]
[[Category:June 2014 in Madrid]]
[[Category:Mariano Rajoy in 2014]]


2017-12-30 22:40:15,020 requests_oauthlib.oauth1_auth DEBUG    Signing request <PreparedRequest [POST]> using client <Client client_key=0a73e346a40b07262b6e36bdba01cba4, client_secret=****, resource_owner_key=5ca4fafa5dd85bb0ecf516d96ad6bf07, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>
2017-12-30 22:40:15,024 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: True
2017-12-30 22:40:15,028 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '58829779440877600631514673615'), ('oauth_timestamp', '1514673615'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73e346a40b07262b6e36bdba01cba4'), ('oauth_token', '5ca4fafa5dd85bb0ecf516d96ad6bf07'), ('titles', 'File:Rajoy junto a los Reyes y los titulares de los poderes del Estado 00.jpg'), ('inprop', 'protect

2017-12-30 22:40:15,283 requests_oauthlib.oauth1_auth DEBUG    Updated body: b'--===============2550444426858795504==\nContent-Type: text/plain\nMIME-Version: 1.0\nContent-disposition: form-data; name="action"\n\nupload\n--===============2550444426858795504==\nContent-Type: text/plain\nMIME-Version: 1.0\nContent-disposition: form-data; name="token"\n\n7a664859006e5c3dc06ae62bb6fbbc785a481521+\\\n--===============2550444426858795504==\nContent-Type: application/octet-stream\nMIME-Version: 1.0\nContent-disposition: form-data; name="text"\nContent-Transfer-Encoding: binary\n\n=={{int:filedesc}}==\n{{Information\n|description={{es|1=El presidente del Gobierno, Mariano Rajoy, saluda al Rey don Juan Carlos, durante el almuerzo que Sus Majestades los Reyes han ofrecido a los titulares de los poderes del Estado.}}\n|date=2014-06-30\n|source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/presidente/Paginas/2014/170614-galeriareyes.aspx Galer\xc3\xada fotogr\xc3\xa1fica: Rajoy junt

2017-12-30 22:40:17,636 pywiki WARNING  API error fileexists-no-change: The upload is an exact duplicate of the current version of [[:File:Rajoy junto a los Reyes y los titulares de los poderes del Estado 00.jpg]].
2017-12-30 22:40:17,639 pywiki VERBOSE  API Error: query=
("{'action': ['upload'], 'token': "
 "['7a664859006e5c3dc06ae62bb6fbbc785a481521+\\\\'], 'text': "
 "['=={{int:filedesc}}==\\n{{Information\\n', 'description={{es', '1=El "
 'presidente del Gobierno, Mariano Rajoy, saluda al Rey don Juan Carlos, '
 'durante el almuerzo que Sus Majestades los Reyes han ofrecido a los '
 "titulares de los poderes del Estado.}}\\n', 'date=2014-06-30\\n', "
 "'source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/presidente/Paginas/2014/170614-galeriareyes.aspx "
 'Galería fotográfica: Rajoy junto a los Reyes y los titulares de los poderes '
 "del Estado]\\n', 'author=Ministerio de la Presidencia. Gobierno de España "
 "(Pool Moncloa)\\n', 'permission=\\n', 'other "
 'versio

Rajoy junto a los Reyes y los titulares de los poderes del Estado 01.jpg
Trying
